In [1]:
import sys
sys.path.append('..')

from utils.dbobjects import *
from utils.utils import *
from utils.config import read_api_key

from urllib.request import urlretrieve

from comicvine import ComicVineClient
from scanners import Vanilla

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError

import requests
import pprint
import functools
import json
import shutil
import pprint as pp
import re
import os

import filecmp
from pathlib import Path
import datetime
import numpy as np
from tqdm.auto import tqdm
from IPython.display import clear_output


In [2]:
api_key = read_api_key()
cv = ComicVineClient(api_key)

class LibraryParser:
    def __init__(self, library_path):
        self.library_path = Path(library_path).resolve()
        self.n_comics = None
        self.n_volumes = None
        self.folder_paths = []
        self.library_dict = {}
        
    def library_watchdog(self):
        pass
    
    def scan_root_folder(self):
        #folders = list({each.parent for each_ext in ['*.cbr', '*.cbz'] for each in self.library_path.rglob(each_ext)})
        
        scanner = Vanilla(library_path=self.library_path)
        self.library_dict = scanner.scan()
        
        
        
#         folddict = {}
#         for each_ext in ['*.cbr', '*.cbz']:
#             for each in self.library_path.rglob(each_ext):
#                 if each.parent.resolve() in folddict:
#                     folddict[each.parent.resolve()].append(each.resolve())
#                 else:
#                     folddict[each.parent.resolve()] = [each.resolve()]
        
#         collection_types = ['Issues', 'TPB', 'Omnibus', 'Annuals']

#         for each in folddict:
#             if any(collection in each.name for collection in collection_types):
#                 for collection in collection_types:
#                     if collection in each.name:
#                         if each.parent.name not in self.library_dict:
#                             self.library_dict[each.parent.name] = {collection.lower() : {'path' : [each], 'assets_in_folder' : folddict[each]}}
                            
#                         else:
#                             if collection.lower() not in self.library_dict[each.parent.name]:
#                                 self.library_dict[each.parent.name].update({collection.lower() : {'path' : [each], 'assets_in_folder' : folddict[each]}})
#                             else:
#                                 #Check here for diffs (Next version update)
#                                 self.library_dict[each.parent.name][collection.lower()]['path'].append(each)
#             else:
#                 self.library_dict[each.name] = {'issues' : {'path' : each, 'assets_in_folder' : folddict[each]}}
        
    def volumify(self):
        for each in self.library_dict:            
             for each_type in self.library_dict[each]:
                self.library_dict[each][each_type]['parser'] = VolumeParser(name=each, path=self.library_dict[each][each_type]['path'], comics_in_folder=self.library_dict[each][each_type]['assets_in_folder'])
    def parse_library(self):
        [self.library_dict[each_volume][each_type]['parser'].parseVolume() for each_volume in self.library_dict.keys() for each_type in self.library_dict[each_volume].keys()]
            

In [6]:
x = LibraryParser('../data/')
x.scan_root_folder()
x.volumify()
pp.pprint(x.library_dict)

{'Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)': {'issues': {'assets_in_folder': [PosixPath('/Users/srihari/Fun/Comics/data/Hulk/Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)/Incredible Hulk 608 (2010) (digital-hd-Empire).cbr'),
                                                                                             PosixPath('/Users/srihari/Fun/Comics/data/Hulk/Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)/Incredible Hulk 600 (2009) (digital-hd-Empire).cbr'),
                                                                                             PosixPath('/Users/srihari/Fun/Comics/data/Hulk/Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)/Incredible Hulk 604 (2010) (digital-hd-Empire).cbr'),
                                                                                             PosixPath('/Users/srihari/Fun/Comics/data/Hulk/Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)/Incredible Hulk 611 (2010) (digital-hd-Empire).cbr'),


In [7]:
pp.pprint(x.library_dict)

{'Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)': {'issues': {'assets_in_folder': [PosixPath('/Users/srihari/Fun/Comics/data/Hulk/Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)/Incredible Hulk 608 (2010) (digital-hd-Empire).cbr'),
                                                                                             PosixPath('/Users/srihari/Fun/Comics/data/Hulk/Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)/Incredible Hulk 600 (2009) (digital-hd-Empire).cbr'),
                                                                                             PosixPath('/Users/srihari/Fun/Comics/data/Hulk/Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)/Incredible Hulk 604 (2010) (digital-hd-Empire).cbr'),
                                                                                             PosixPath('/Users/srihari/Fun/Comics/data/Hulk/Incredible Hulk 600-611 (2009-2010) (digital-hd-Empire)/Incredible Hulk 611 (2010) (digital-hd-Empire).cbr'),


In [4]:
class ComicParser(): #Parser object to extract name, first year and issue number.
    def __init__(self, path):
        self.comic_name = None
        self.comic_year = None
        self.issue_number = None
        self.comic_path = path
        self.comic_metadata = {}
        self.comic_initiated = False

    def parseComic(self):
        self.comic_name = os.path.splitext(os.path.basename(self.comic_path))[0]
        years = re.findall(r'\(([12]\d{3})', self.comic_name)
        if years:
            year_within_brackets = years[0]
            if (len(year_within_brackets) == 4): #Stupid way of doing but good for now.
                self.comic_year = int(year_within_brackets)
            else:
                self.comic_year = 0
        self.issue_number = re.findall(r"(\d+\.?\d?[a-zA-Z]{0,3}?)(?:\s*\(of|\s*\([12]\d{3}\))", self.comic_path) #Get the digits or digits with point number (19.1) or digits followed by letters (19.INH) either before "(of" or before a year. 

In [5]:
class VolumeParser:
    def __init__(self, name=None, path=None, comics_in_folder=None):
        self.volume_name = name
        self.volume_path = path
        self.volumeDict = {'comics_in_folder' : comics_in_folder } 
        
        
        self.volume_initiated = False
        self.update_index = 0
        self.volumeTable = None
        
        
        self.searchReturned = None
        self.HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:7.0) '
                     'Gecko/20130825 Firefox/36.0'}
        self.params = {'api_key':api_key, 'format': 'json'} 
                
    def parseVolume(self): #Adds path and volume name to the parser object. Also parses comic. 
        """
        Need to make sure, only if the parser is not initiated by a library, we need to pathlib actions here. We can use None passes.
        """
        
        
        
        comic_dir = self.volume_path
        if '(' in self.volume_name:
            #elf.volume_name = os.path.basename(self.volume_path).split(' (')[0] 
            self.volume_name = self.volume_name.split(' (')[0].strip()
        else:    
            #self.volume_name = os.path.basename(self.volume_path) 
            self.volume_name = self.volume_name
        
        print(self.volume_name)
#         self.volumeDict['from_file_volume_name'] = self.volume_name        
#         for entry in os.listdir(comic_dir):
#             if not entry.startswith(".") and check_comic(entry):
#                 comic = ComicParser(os.path.join(comic_dir,entry)) #convert to path join volume path and entry
#                 comic.parseComic()            
#                 self.volumeDict['comics_in_folder'].update({comic.comic_name : comic})
                                
#         print(str(len(self.volumeDict['comics_in_folder']))+' comics added in volume ' + self.volume_name +'.')    
        
    def searchQuery(self,api_key):
        """
        Search Comic Vine Servers
        """
        print("Fetching from Comic Vine servers")
        years = [self.volumeDict['comics_in_folder'][key].comic_year for key in self.volumeDict['comics_in_folder'].keys() if self.volumeDict['comics_in_folder'][key].comic_year]        
        searchtext = self.volumeDict['from_file_volume_name']
        if years:
            searchtext += (' '+str(years[0]))
        print('Querying for '+searchtext)
        firstresponse = cv.search(searchtext, resources=['volume']) #Search for volume name with year
        self.searchReturned = firstresponse.results
        
    def findBestSearch(self):
        """
        Finding best search among Volumes. 2 main information considered 
        here are number of assets in folder folder (which is assumed to be 
        total number of issues in that volume) and year range obtained 
        from comic file names.        
        
        First Result | No. of Issues | Year | Score | Result
              o               o          o      80      o
              o               X          o      70      o
              X               o          o      50      o
              o               o          X      40      X -
              X               X          o      40      X |
              o               X          X      30      X |-- Fuzzy search for these guys
              X               o          X      10      X -
        
        """
        print("Finding best match")
        expected_num_comics = len(self.volumeDict['comics_in_folder'])    
        existing_comics = list(self.volumeDict['comics_in_folder'].keys())
        years = [self.volumeDict['comics_in_folder'][key].comic_year for key in self.volumeDict['comics_in_folder'].keys() if self.volumeDict['comics_in_folder'][key].comic_year]
        
        
        flag=False
        best_index = 0
        for indx, each_result in enumerate(self.searchReturned):    
            if indx is 0:
                each_result['match_score'] = 30.0 #First result returned is the best result.
            elif indx is not 0:
                each_result['match_score'] = 0.0
            if expected_num_comics == each_result['count_of_issues']:
                each_result['match_score'] += 10.0 
            if years:    
                if (min(years)<=int(each_result['start_year'])<=max(years)):
                    each_result['match_score'] +=40.0
                          
            if flag and each_result['match_score']<=40:
                detailresponse = requests.get(each_result['api_detail_url'], headers=self.HEADERS, params=self.params)
                titles_in_vol = [each['name'] for each in detailresponse.json()['results']['issues']]
                each_result['match_score'] +=fuzz.token_set_ratio(titles_in_vol, existing_comics)
            
                    
            if each_result['match_score']>=np.array(self.searchReturned)[best_index]['match_score']: #Also assumed that search results are coming back sorted in relavance.
                best_index = indx
        self.volumeDict['best_search']=np.array(self.searchReturned)[best_index]
        print('Match Score : '+ str(self.volumeDict['best_search']['match_score']))        
    
    def confirmResult(self):
        print_com_meta(self.volumeDict['best_search'], 'Best Result:')
        confirmation = query_yes_no("Is the information correct?")
        choice = None
        while True:
            clear_output()
            if not confirmation: 
                for idx, each in enumerate(self.searchReturned):
                    print_com_meta(each, idx+1)
                choice = sanitised_input("Please make a manual choice between 1-10 : ", int, 1,10)
                print_com_meta(self.searchReturned[choice-1], 'Your Choice : ')
                confirmation = query_yes_no("Is the information correct?")
            else:
                if choice:
                    self.volumeDict['best_search'] = self.searchReturned[choice-1]
                break
    
    def fetchVolumeMetadata(self):                                        
        detailresponse = requests.get(self.volumeDict['best_search']['api_detail_url'], headers=self.HEADERS, params=self.params)
        fetched_volume = detailresponse.json()
        self.volumeDict['detailed_meta'] = fetched_volume['results']        
        #Image Directory
        orig_directory = './Resources/Images/VolumeArt/'
        img_name = os.path.basename(self.volumeDict['detailed_meta']['image']['original_url'])
        img_path = os.path.join(orig_directory,img_name)
        #Fetch Image
        urlretrieve(self.volumeDict['detailed_meta']['image']['original_url'], img_path)        
        self.volume_initiated = True
        volume = Volume(
                        id = self.volumeDict['detailed_meta']['id'],
                        name = self.volumeDict['detailed_meta']['name'],
                        aliases = self.volumeDict['detailed_meta']['aliases'],
                        count_of_issues = self.volumeDict['detailed_meta']['count_of_issues'],
                        date_added = self.volumeDict['detailed_meta']['date_added'],
                        date_last_updated = self.volumeDict['detailed_meta']['date_last_updated'],
                        deck = self.volumeDict['detailed_meta']['deck'],
                        description = re.sub(r'<[^>]*>', '', self.volumeDict['detailed_meta']['description']), #remove all within angle brackets
                        publisher = self.volumeDict['detailed_meta']['publisher']['name'], #Later to be changed to publisher object
                        start_year = self.volumeDict['detailed_meta']['start_year'],
                        comicvine_api_detail_url = self.volumeDict['detailed_meta']['api_detail_url'],
                        comicvine_image = self.volumeDict['detailed_meta']['image']['original_url'], #image
                        comicvine_site_detail_url = self.volumeDict['detailed_meta']['site_detail_url'],
                        local_path = self.volume_path,
                        local_image_path = img_path,
                        #character_credits = Column(String)
                        #concept_credits = Column(String)
                        #team_credits = Column(String)
                        #location_credits = Column(String)
                        #object_credits = Column(String)
                        #person_credits = Column(String)
                        )
        self.volumeTable = volume        
        
    def fetchComicsMetadata(self, api_key):
        comic_fetch_loop = tqdm(self.volumeDict['detailed_meta']['issues'])
        comicObjects = []
        issue_directory = './Resources/Images/IssueArt/'                        
        for each_issue in comic_fetch_loop:
            per_issue_response = requests.get(each_issue['api_detail_url'], headers=self.HEADERS, params=self.params).json()['results']
            img_name = os.path.basename(per_issue_response['image']['original_url'])
            img_path = os.path.join(issue_directory,img_name)
            #Fetch Image if doesnt exist
            if not os.path.isfile(img_path):
                urlretrieve(per_issue_response['image']['original_url'], img_path) 
            
            comic = Comic(id = per_issue_response['id'],
                          name = per_issue_response['name'],
                          aliases = per_issue_response['aliases'],
                          deck = per_issue_response['deck'],
                          description = re.sub(r'<[^>]*>', '', per_issue_response['description']),
                          cover_date = per_issue_response['cover_date'],
                          date_added = per_issue_response['date_added'],
                          date_last_updated = per_issue_response['date_last_updated'],
                          issue_number = per_issue_response['issue_number'],
                          comicvine_api_detail_url = per_issue_response['api_detail_url'],
                          comicvine_image = per_issue_response['image']['original_url'],
                          local_image_path = img_path
            )
            comicObjects.append(comic)
            comic_fetch_loop.set_postfix(Status='Added '+str(comic.name))            
        self.volumeTable.comics=set(comicObjects)
     
    def commit_to_db(self, dbpath='sqlite:///comicdb.db'):
        
        app = Flask(__name__)
        app.config['SQLALCHEMY_DATABASE_URI'] = dbpath
        db = SQLAlchemy(app)
        
        
        engine = create_engine(dbpath, echo=False)
        Base.metadata.create_all(engine)
        Session = sessionmaker(bind=engine)
        session = Session()
        session.add(self.volumeTable)
        
        try:
            session.commit()
        except IntegrityError:
            session.rollback()
        
        session.close()
        engine.dispose()

            
#             for key in self.volumeDict['comics_in_folder'].keys():
#                 if self.volumeDict['comics_in_folder'][key].issue_number == per_issue_response['issue_number'] and self.volumeDict['comics_in_folder'][key].comic_initiated == False:
#                     self.volumeDict['comics_in_folder'][key].comic_metadata = per_issue_response
#                     self.volumeDict['comics_in_folder'][key].comic_initiated = True

In [ ]:
x.library_dict['Infinity Gauntlet']['tpb']['volume_parser']

In [ ]:
vol1 = VolumeParser('./data/Infinity Gauntlet')
vol1.parseVolume()
vol1.searchQuery(api_key)
vol1.findBestSearch()
vol1.confirmResult()
#vol1.fetchVolumeMetadata()

In [ ]:
vol1.volumeDict['best_search']

In [ ]:
vol1.fetchVolumeMetadata()

In [ ]:
vol1.fetchComicsMetadata(api_key)

In [ ]:
vol1.commit_to_db()

In [ ]:
subs = {subdir:files for subdir, dirs, files in os.walk('./data') for name in files if name.endswith(('cbr', 'cbz'))}

In [ ]:
list(subs.keys())
#subs['./data/Thor/Thor Vol. 2 (1998-2004)/Issues']

In [ ]:
vol1.volumeDict

In [ ]:
mainp = Path('./data')

In [ ]:
folders = list({each.parent for each_ext in ['*.cbr', '*.cbz'] for each in mainp.rglob(each_ext)})
folders

In [ ]:
list(folders[0].glob('*'))[0].name

In [ ]:
to_search = []
for each in folders:
    if 'Issue' in each.name or 'TPB' in each.name:
        to_search.append(each.parent.name)
    else:
        to_search.append(each.name)
to_search